***

# Customer Recommendation System

***

Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

***

## Datasets

Shops

In [57]:
shops = pd.read_csv("data/shops.csv")

In [58]:
shops.head()

,shopID,Name,Food,Clothing,Furniture,Electronics,Accessories,Grocery,Cosmetics,Toys
0,1,Papa Johns,1,0,0,0,0,0,0,0
1,2,Coldstone,1,0,0,0,0,0,0,0
2,3,BrewBakes,1,0,0,0,0,0,0,0
3,4,OCD,1,0,0,0,0,0,0,0
4,5,Lassi Shop,1,0,0,0,0,0,0,0


***

Users

In [61]:
users = pd.read_csv("data/users.csv")
users.head()

,Unnamed: 0,userID,FirstName,LastName,EmailID,Password,Phone,Age
0,0,12345,Sarthak,Singh,sarthak.singh2017@vitstudent.ac.in,sarthak,NaN,20
1,1,12346,Ipshita,Joshi,ipshitamjoshi@gmail.com,ipshita,NaN,20
2,2,12347,Sangat,Baral,sangatbaral@gmail.com,sangat,NaN,20
3,3,12348,User1,NaN,NaN,NaN,NaN,20
4,4,12349,User2,NaN,NaN,NaN,NaN,20


In [5]:
itr = 12345
useritr = 1
for index, rows in users.iterrows():
    users.loc[index, 'UserID'] = itr
    if itr>12347:
        users.loc[index,'FirstName'] = 'User{}'.format(useritr)
        useritr += 1 
    itr += 1

In [60]:
users.head()

,Unnamed: 0,userID,FirstName,LastName,EmailID,Password,Phone,Age
0,0,12345,Sarthak,Singh,sarthak.singh2017@vitstudent.ac.in,sarthak,NaN,20
1,1,12346,Ipshita,Joshi,ipshitamjoshi@gmail.com,ipshita,NaN,20
2,2,12347,Sangat,Baral,sangatbaral@gmail.com,sangat,NaN,20
3,3,12348,User1,NaN,NaN,NaN,NaN,20
4,4,12349,User2,NaN,NaN,NaN,NaN,20


In [7]:
users.to_csv("data/users.csv")

Ratings

In [114]:
rating = pd.read_csv('data/ratings.csv')
rating.head()

,userID,shopID,ratings
0,12345,3,3.5
1,12345,8,4.0
2,12345,10,4.0
3,12345,12,4.5
4,12345,24,3.5


***

In [116]:
user_ratings = rating.groupby(by='userID', as_index=False)['ratings'].mean()
# Ratings.groupby(by="userId",as_index=False)['rating'].mean()

In [117]:
user_ratings

,userID,ratings
0,12345,3.407407
1,12346,3.461538
2,12347,3.234375
3,12348,3.455882
4,12349,3.090909


In [ ]:
ratings = rating.copy()

In [118]:
ratings['user_avg'] = 0
for index, rows in ratings.iterrows():
    ratings.loc[index, 'avg_rating'] = user_ratings[ratings.loc[index, 'userID']]

KeyError: '12345'

In [112]:
ratings['norm_rating'] = ratings['ratings'] - ratings['avg_rating']

In [113]:
ratings.head()

,userID,shopID,ratings,user_avg,avg_rating,norm_rating
0,12345,3,3.5,0,3.407407,0.092593
1,12345,8,4.0,0,3.407407,0.592593
2,12345,10,4.0,0,3.407407,0.592593
3,12345,12,4.5,0,3.407407,1.092593
4,12345,24,3.5,0,3.407407,0.092593


In [67]:
final = pd.pivot_table( ratings, values='norm_rating', index='userID',columns='shopID')
final.head()

shopID,1,2,3,4,5,6,7,8,9,10,...,61,62,63,64,65,66,67,68,69,70
userID,,,,,,,,,,,,,,,,,,,,,
12345,NaN,NaN,0.092593,-0.407407,NaN,NaN,NaN,0.592593,NaN,0.592593,...,-1.407407,NaN,1.092593,0.592593,-0.407407,NaN,NaN,NaN,NaN,NaN
12346,NaN,NaN,NaN,NaN,0.538462,NaN,NaN,NaN,NaN,NaN,...,NaN,0.038462,-1.461538,NaN,1.038462,0.038462,0.038462,0.538462,NaN,0.038462
12347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.734375,-1.234375,NaN,...,NaN,0.765625,-1.234375,0.265625,-0.234375,NaN,NaN,-0.234375,-0.734375,NaN
12348,0.544118,0.544118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.455882,-1.455882,1.544118,0.544118,NaN,0.044118,0.544118,NaN,NaN,NaN
12349,NaN,NaN,0.909091,-0.090909,-1.090909,-1.090909,-0.090909,NaN,-0.590909,NaN,...,-0.590909,NaN,0.909091,0.409091,-0.090909,-1.090909,NaN,-0.590909,-1.090909,NaN


In [68]:
#replacing by shop
final_shop = final.fillna(final.mean(axis=0))

#replacing by user average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [69]:
final_shop.head()

shopID,1,2,3,4,5,6,7,8,9,10,...,61,62,63,64,65,66,67,68,69,70
userID,,,,,,,,,,,,,,,,,,,,,
12345,0.544118,0.544118,0.092593,-0.407407,-0.276224,-1.090909,-0.090909,0.592593,-0.912642,0.592593,...,-1.407407,-0.217265,1.092593,0.592593,-0.407407,-0.336110,0.291290,-0.095608,-0.912642,0.038462
12346,0.544118,0.544118,0.500842,-0.249158,0.538462,-1.090909,-0.090909,-0.070891,-0.912642,0.592593,...,-1.151400,0.038462,-1.461538,0.452857,1.038462,0.038462,0.038462,0.538462,-0.912642,0.038462
12347,0.544118,0.544118,0.500842,-0.249158,-0.276224,-1.090909,-0.090909,-0.734375,-1.234375,0.592593,...,-1.151400,0.765625,-1.234375,0.265625,-0.234375,-0.336110,0.291290,-0.234375,-0.734375,0.038462
12348,0.544118,0.544118,0.500842,-0.249158,-0.276224,-1.090909,-0.090909,-0.070891,-0.912642,0.592593,...,-1.455882,-1.455882,1.544118,0.544118,0.076443,0.044118,0.544118,-0.095608,-0.912642,0.038462
12349,0.544118,0.544118,0.909091,-0.090909,-1.090909,-1.090909,-0.090909,-0.070891,-0.590909,0.592593,...,-0.590909,-0.217265,0.909091,0.409091,-0.090909,-1.090909,0.291290,-0.590909,-1.090909,0.038462


In [70]:
final_user.head()

shopID,1,2,3,4,5,6,7,8,9,10,...,61,62,63,64,65,66,67,68,69,70
userID,,,,,,,,,,,,,,,,,,,,,
12345,-3.289550e-17,-3.289550e-17,9.259259e-02,-4.074074e-01,-3.289550e-17,-3.289550e-17,-3.289550e-17,5.925926e-01,-3.289550e-17,5.925926e-01,...,-1.407407e+00,-3.289550e-17,1.092593,5.925926e-01,-0.407407,-3.289550e-17,-3.289550e-17,-3.289550e-17,-3.289550e-17,-3.289550e-17
12346,-1.366428e-16,-1.366428e-16,-1.366428e-16,-1.366428e-16,5.384615e-01,-1.366428e-16,-1.366428e-16,-1.366428e-16,-1.366428e-16,-1.366428e-16,...,-1.366428e-16,3.846154e-02,-1.461538,-1.366428e-16,1.038462,3.846154e-02,3.846154e-02,5.384615e-01,-1.366428e-16,3.846154e-02
12347,7.560484e-03,7.560484e-03,7.560484e-03,7.560484e-03,7.560484e-03,7.560484e-03,7.560484e-03,-7.343750e-01,-1.234375e+00,7.560484e-03,...,7.560484e-03,7.656250e-01,-1.234375,2.656250e-01,-0.234375,7.560484e-03,7.560484e-03,-2.343750e-01,-7.343750e-01,7.560484e-03
12348,5.441176e-01,5.441176e-01,-1.648841e-02,-1.648841e-02,-1.648841e-02,-1.648841e-02,-1.648841e-02,-1.648841e-02,-1.648841e-02,-1.648841e-02,...,-1.455882e+00,-1.455882e+00,1.544118,5.441176e-01,-0.016488,4.411765e-02,5.441176e-01,-1.648841e-02,-1.648841e-02,-1.648841e-02
12349,8.074349e-17,8.074349e-17,9.090909e-01,-9.090909e-02,-1.090909e+00,-1.090909e+00,-9.090909e-02,8.074349e-17,-5.909091e-01,8.074349e-17,...,-5.909091e-01,8.074349e-17,0.909091,4.090909e-01,-0.090909,-1.090909e+00,8.074349e-17,-5.909091e-01,-1.090909e+00,8.074349e-17


In [71]:
# user similarity on final_user
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0)
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userID,12345,12346,12347,12348,12349
userID,,,,,
12345,0.000000,-0.433816,-0.362079,0.507782,0.026562
12346,-0.433816,0.000000,0.268494,-0.311959,0.009404
12347,-0.362079,0.268494,0.000000,-0.146341,0.094105
12348,0.507782,-0.311959,-0.146341,0.000000,0.068027
12349,0.026562,0.009404,0.094105,0.068027,0.000000


In [72]:
# user similarity on final_shop
cosine = cosine_similarity(final_shop)
np.fill_diagonal(cosine, 0 )
similarity_with_shop = pd.DataFrame(cosine,index=final_shop.index)
similarity_with_shop.columns=final_user.index
similarity_with_shop.head()

userID,12345,12346,12347,12348,12349
userID,,,,,
12345,0.000000,0.359618,0.352146,0.739193,0.614723
12346,0.359618,0.000000,0.622061,0.406958,0.543297
12347,0.352146,0.622061,0.000000,0.422703,0.559206
12348,0.739193,0.406958,0.422703,0.000000,0.659753
12349,0.614723,0.543297,0.559206,0.659753,0.000000


In [73]:
def n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [74]:
# top 30 neighbours for each user
sim_user_30_u = n_neighbours(similarity_with_user,5)
sim_user_30_u.head()

,top1,top2,top3,top4,top5
userID,,,,,
12345,12348,12349,12345,12347,12346
12346,12347,12349,12346,12348,12345
12347,12346,12349,12347,12348,12345
12348,12345,12349,12348,12347,12346
12349,12347,12348,12345,12346,12349


In [75]:
# top 30 neighbours for each user
sim_user_30_m = n_neighbours(similarity_with_shop,5)
sim_user_30_m.head()

,top1,top2,top3,top4,top5
userID,,,,,
12345,12348,12349,12346,12347,12345
12346,12347,12349,12348,12345,12346
12347,12346,12349,12348,12345,12347
12348,12345,12349,12347,12346,12348
12349,12348,12345,12347,12346,12349


In [76]:
def similar_shops(user1, user2):
    common_shops = ratings[ratings['userID'] == user1].merge(ratings[ratings['userID'] == user2], 
                                                             on = "shopID", how = "inner" )
    return common_shops.merge( shops, on = 'shopID' )

In [85]:
a = similar_shops(12349,12348)
a = a.loc[ : , ['ratings_x','ratings_y','Name']]
a.head()

,ratings_x,ratings_y,Name
0,3.5,4.5,Taco Bell
1,3.5,4.0,Dunkin Doughnuts
2,2.0,3.0,Fizzy Restaurant
3,4.0,3.0,Splash
4,4.0,2.5,Debenhams


In [86]:
a

,ratings_x,ratings_y,Name
0,3.5,4.5,Taco Bell
1,3.5,4.0,Dunkin Doughnuts
2,2.0,3.0,Fizzy Restaurant
3,4.0,3.0,Splash
4,4.0,2.5,Debenhams
5,3.0,4.5,Louis Vuitton
6,4.0,4.5,Gucci
7,4.0,3.5,Gucci
8,4.0,5.0,Apple Store
9,3.5,4.0,Bose


In [97]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_shop.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = user_ratings.loc[user_ratings['userID'] == user,'ratings'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_shop.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [99]:
score = User_item_score(12345,63)
print("score (u,i) is",score)

score (u,i) is 3.7656257110265305


In [100]:
ratings = ratings.astype({"userID": str})
shop_user = ratings.groupby(by = 'userID')['shopID'].apply(lambda x:','.join(x))

TypeError: sequence item 0: expected str instance, int found

In [108]:
ratings.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'